In [1]:
import os
import glob
from torch.utils.data import DataLoader
from custom_dataset_cross import MyDataset
import cv2
from share import *
import torchvision

from pytorch_lightning import seed_everything

os.environ["CUDA_VISIBLE_DEVICES"]="4"


seed = 42 
if False:
    seed = random.randint(0, 65535)
seed_everything(seed)


dataset = MyDataset("kin_hed2")
dataloader = DataLoader(dataset, num_workers=1, batch_size=1, shuffle=True)


def get_key(name: str):
    name = name.split(".")[1]
    name = name.split("_")[-1]
    name = name[3:]
    return int(name)


def get_sequence():
    for batch in dataloader:
        meta_batch = batch["meta"]["file_name"]
        meta = meta_batch[0]
        meta = meta.split("/")[1]
        meta = meta.split("_")[:-1]
        meta = "_".join(meta)
        print(meta)
        styles = glob.glob("./data/kin_hed2/jpg/" + meta + "*")
        styles.sort(key=get_key)
        structures = glob.glob("./data/kin_hed2/hint/" + meta + "*")
        structures.sort(key=get_key)
        print(styles)
        print(structures)
        return styles, structures


def get_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = img.astype(np.float32) / 255.0
    return img

logging improved.


/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
Global seed set to 42


In [2]:
import einops
import numpy as np
from ldm.modules.encoders.modules import FrozenClipImageEmbedder
import torch.nn.functional as F
import config
import gradio as gr
import random
import torch
from PIL import Image
from annotator.util import resize_image, HWC3
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler

In [3]:
p = "./train_log/kin_hed_dropout1/lightning_logs/version_1/checkpoints/epoch=6-step=595385.ckpt"
# p = "./train_log/kin_hed_2/lightning_logs/version_0/checkpoints/epoch=1-step=119930.ckpt"
# p = './train_log/kin_hed_cross_3/lightning_logs/version_0/checkpoints/epoch=4-step=412223.ckpt'

model = create_model("./models/cldm_v15_cross.yaml").cpu()
model.load_state_dict(load_state_dict(p, location="cuda"))
model = model.cuda()
ddim_sampler = DDIMSampler(model)
frozenClipImageEmbedder = model.style_encoder

if config.save_memory:
    model.low_vram_shift(is_diffusing=False)

/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/distributed/_sharded_tensor/__init__.py:8: DeprecationWarning: torch.distributed._sharded_tensor will be deprecated, use torch.distributed._shard.sharded_tensor instead
  warnings.warn(


ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_

Global seed set to 42


In [39]:
ddim_steps = 50
strength = 1
eta = 0
scale = 3


grid = []
for i in range(5):
    styles, structures = get_sequence()
    styles = np.array([get_img(style) for style in styles])
    styles = styles.copy().astype(np.float32)

    structures = np.array([get_img(structure) for structure in structures])
    structures = structures.astype(np.float32) / 255.0

    B, H, W, C = structures.shape

    # we only want to take the space information from the first image
    styles_first = np.repeat(styles[[0]], B, axis=0)

    control = torch.from_numpy(structures.copy()).float().cuda()
    control = einops.rearrange(control, "b h w c -> b c h w")

    style_embedding = frozenClipImageEmbedder(styles_first)

    c_style = style_embedding.last_hidden_state
    c_embed = style_embedding.pooler_output
    c_prompt = model.get_learned_conditioning(
        ["a professional, detailed, high-quality image"] * B
    )

    uc_style = torch.zeros_like(c_style)
    uc_embed = torch.zeros_like(c_embed)
    uc_prompt = model.get_learned_conditioning(
        ["a professional, detailed, high-quality image"] * B
    )

    cond = {
        "c_concat": [control],
        "c_crossattn": [c_prompt],
        "c_style": [c_style],
        "c_embed": [c_embed],
    }
    un_cond = {
        "c_concat": [control],
        "c_crossattn": [uc_prompt],
        "c_style": [uc_style],
        "c_embed": [uc_embed],
    }
    shape = (4, H // 8, W // 8)

    model.control_scales = [
        strength
    ] * 13  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
    samples, intermediates = ddim_sampler.sample(
        ddim_steps,
        B,
        shape,
        cond,
        verbose=False,
        eta=eta,
        unconditional_guidance_scale=scale,
        unconditional_conditioning=un_cond,
    )

    d_samples = model.decode_first_stage(samples)
    x_samples = einops.rearrange(d_samples, "b c h w -> h (b w) c") * 127.5 + 127.5
    x_samples = x_samples.cpu().numpy().clip(0, 255).astype(np.uint8)
    x_samples = np.concatenate(
        [np.zeros((512, 512, 3), dtype=np.uint8), x_samples], axis=1
    )

    f_styles = (
        einops.rearrange(styles, "b h w c -> h (b w) c").clip(0, 255).astype(np.uint8)
    )
    final = np.concatenate([f_styles, x_samples], axis=0)

    grid.append(final)

vWd1q37V6eY_000054_000064
['./data/kin_hed2/jpg/vWd1q37V6eY_000054_000064_idx0.png', './data/kin_hed2/jpg/vWd1q37V6eY_000054_000064_idx500.png', './data/kin_hed2/jpg/vWd1q37V6eY_000054_000064_idx1000.png', './data/kin_hed2/jpg/vWd1q37V6eY_000054_000064_idx1500.png', './data/kin_hed2/jpg/vWd1q37V6eY_000054_000064_idx2000.png']
['./data/kin_hed2/hint/vWd1q37V6eY_000054_000064_idx500.png', './data/kin_hed2/hint/vWd1q37V6eY_000054_000064_idx1000.png', './data/kin_hed2/hint/vWd1q37V6eY_000054_000064_idx1500.png', './data/kin_hed2/hint/vWd1q37V6eY_000054_000064_idx2000.png']
Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 50 timesteps


DDIM Sampler:  88%|████████▊ | 44/50 [00:18<00:02,  2.40it/s]

In [ ]:
f_grid = np.concatenate(grid, axis=0)

Image.fromarray(f_grid).save("grid.png")

In [28]:
num_sanity_val_steps

(5120, 2560, 3)